In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'asl-signs:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F46105%2F5087314%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240423%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240423T005456Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D05de445137be291fccc16e31846b960ae442369be6b9cf76c85d5982dc27fd1c5dc4f9cfd2ded022a600781a52b00cfee68e542bb41dfda85082af356e8d746d5841f6d5f759649c931aebe0935a25026cdc36fd4352c8e285ff10ebca3f422b5928426de9bccd0205651e6553eb9c19caab051ff92390653248efe87c53bd4d13047dddce5e7e904c60b047de829dd928ea3b7008ac7bf5453ec669f5808d85d12eb32f86a61b4ee98b056fcb8ff80bdae6039103aa2fef7289547b942d309dd9e1f14c346a0d8700e6f7988911a44dbc63ea1c8a3a52d12ef8835f3946e6732bab3c4e861835a15178d6886cabbbdcf45222311e8fe7a50bb41ffb832b29ba'

KAGGLE_INPUT_PATH='kaggle/input'
KAGGLE_WORKING_PATH='kaggle/working'
KAGGLE_SYMLINK='kaggle'

# os.makedirs(KAGGLE_SYMLINK)
# os.makedirs(KAGGLE_INPUT_PATH, 0o777)
# os.makedirs(KAGGLE_WORKING_PATH, 0o777)

# for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
#     directory, download_url_encoded = data_source_mapping.split(':')
#     download_url = unquote(download_url_encoded)
#     filename = urlparse(download_url).path
#     destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
#     try:
#         with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
#             total_length = fileres.headers['content-length']
#             print(f'Downloading {directory}, {total_length} bytes compressed')
#             dl = 0
#             data = fileres.read(CHUNK_SIZE)
#             while len(data) > 0:
#                 dl += len(data)
#                 tfile.write(data)
#                 done = int(50 * dl / int(total_length))
#                 sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
#                 sys.stdout.flush()
#                 data = fileres.read(CHUNK_SIZE)
#             if filename.endswith('.zip'):
#               with ZipFile(tfile) as zfile:
#                 zfile.extractall(destination_path)
#             else:
#               with tarfile.open(tfile.name) as tarfile:
#                 tarfile.extractall(destination_path)
#             print(f'\nDownloaded and uncompressed: {directory}')
#     except HTTPError as e:
#         print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
#         continue
#     except OSError as e:
#         print(f'Failed to load {download_url} to path {destination_path}')
#         continue

# print('Data source import complete.')


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [64]:
from glob import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import json
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OneHotEncoder



In [5]:
NUM_FRAMES=60

In [6]:
data_files = glob("kaggle/input/asl-signs/train_landmark_files/*/*.parquet", recursive=True)


In [7]:
example_pq_df = pd.read_parquet(data_files[1])
first_frame_df = example_pq_df[example_pq_df['frame'] == example_pq_df['frame'].iloc[0]]
first_frame_head_df = first_frame_df[first_frame_df['type'] == 'face']
first_frame_pose_df = first_frame_df[first_frame_df['type'] == 'pose']
first_frame_left_hand_df = first_frame_df[first_frame_df['type'] == 'left_hand']
first_frame_right_hand_df = first_frame_df[first_frame_df['type'] == 'right_hand']

print(example_pq_df.head())
print("landmarks per frame", len(first_frame_df))
print("face landmarks ", len(first_frame_head_df))
print("pose landmarks ", len(first_frame_pose_df))
print("left hand landmarks ", len(first_frame_left_hand_df))
print("right hand landmarks ", len(first_frame_right_hand_df))

train_labels = [file.split('/')[4] for file in data_files]

print("Number of labels: ", train_labels)



   frame    row_id  type  landmark_index         x         y         z
0      9  9-face-0  face               0  0.520766  0.544308 -0.045363
1      9  9-face-1  face               1  0.509756  0.507224 -0.059449
2      9  9-face-2  face               2  0.512931  0.520984 -0.039289
3      9  9-face-3  face               3  0.503155  0.481343 -0.032631
4      9  9-face-4  face               4  0.509599  0.497557 -0.059698
landmarks per frame 543
face landmarks  468
pose landmarks  33
left hand landmarks  21
right hand landmarks  21
Number of labels:  ['4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4718', '4

In [110]:
'''
Methods to either upsample or downsample frames to return NUM_FRAMES freams
'''
def interpolate_frames(pq_df, num_frames=NUM_FRAMES):
    current_frames = pq_df['frame'].unique()
    start_frame = current_frames[0] 
    needed_frames = num_frames - len(current_frames)
    frame_interval = len(current_frames) // (needed_frames + 1)
    for i in range(1, needed_frames + 1):
        frame = start_frame + i * frame_interval % len(pq_df['frame'].unique())
        end_index_of_first_half = len(pq_df[pq_df['frame'] <= frame])
        pq_df = pd.concat([pq_df[pq_df['frame'] <= frame], pq_df[pq_df['frame'] >= frame]], )
        pq_df = pq_df.reset_index(drop=True)
        pq_df.loc[pq_df.index >= end_index_of_first_half, 'frame'] = pq_df.loc[pq_df.index >= end_index_of_first_half, 'frame'] + 1
    for index, frame in enumerate(pq_df['frame'].unique()):
        pq_df.loc[pq_df['frame'] == frame, 'frame'] = index
    return pq_df

def extract_frames(pq_df, method='uniform' , num_frames=NUM_FRAMES):  
    if method == 'uniform':
        total_frames = len(pq_df['frame'].unique())
        first_frame = pq_df['frame'].iloc[0]
        step_size = total_frames // num_frames
        frame_indices_range = range(first_frame, total_frames+first_frame, step_size)
    elif method == 'end':
        unique_frames = pq_df['frame'].unique()
        frame_indices_range = unique_frames[-num_frames:]
    elif method == 'start':
        unique_frames = pq_df['frame'].unique()
        frame_indices_range = unique_frames[:num_frames]
    frame_indices = list(frame_indices_range)[-num_frames:]
    new_df = pq_df[pq_df['frame'].isin(frame_indices)].copy()
    for index, frame in enumerate(frame_indices):
        new_df.loc[new_df['frame'] == frame, 'frame'] = index
    return new_df

interpolate_frames_df = interpolate_frames(pd.read_parquet(data_files[4]))
print(interpolate_frames_df.frame.unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59]


In [114]:
def onehot_encode(data, num_classes):
    num_labels = num_classes
    one_hot = np.zeros((len(data), num_labels), dtype=int)
    one_hot[np.arange(len(data)), data] = 1
    return one_hot

def labels_mapping(csv_file, key_column, value_column):
    df = pd.read_csv(csv_file)
    df[key_column] = df[key_column].astype(str)
    dictionary = df.set_index(key_column)[value_column].to_dict()
    return dictionary

def sample_parquets(data_file, n=10000):
    indices = np.random.choice(len(data_file), n)
    labels_mapping_dict = labels_mapping('kaggle/input/asl-signs/train.csv', 'sequence_id', 'sign')
    data = [data_file[i] for i in indices]
    labels = [labels_mapping_dict[d.split('/')[-1].split('.')[0]] for d in data]
    label_indexes = convert_labels_to_indexes(labels)
    one_hot_labels = onehot_encode(label_indexes, 250)
    return data, one_hot_labels

def convert_labels_to_indexes(labels):
    json.load(open('kaggle/input/asl-signs/sign_to_prediction_index_map.json'))
    return [json.load(open('kaggle/input/asl-signs/sign_to_prediction_index_map.json'))[label] for label in labels]

# labels_mapping_dict = labels_mapping('kaggle/input/asl-signs/train.csv', 'sequence_id', 'sign')

(10, 250)


In [115]:
def process_data(data_files):
    df_lists = []
    for i, data  in enumerate(data_files):
        data_df = pd.read_parquet(data)
        new_df = None
        if(i % 100 == 0):
            print(i)
        if len(data_df['frame'].unique()) < NUM_FRAMES:
            new_df = interpolate_frames(data_df)
        elif len(data_df['frame'].unique() > NUM_FRAMES):
            new_df = extract_frames(data_df)
        df_lists.append(new_df)
    return df_lists

            
            
            


In [76]:
print("Processing data", sample_parquets(data_files, 10))

Processing data ['kaggle/input/asl-signs/train_landmark_files/2044/3223383383.parquet', 'kaggle/input/asl-signs/train_landmark_files/36257/2016094763.parquet', 'kaggle/input/asl-signs/train_landmark_files/49445/51906650.parquet', 'kaggle/input/asl-signs/train_landmark_files/32319/1656119234.parquet', 'kaggle/input/asl-signs/train_landmark_files/37055/1943676600.parquet', 'kaggle/input/asl-signs/train_landmark_files/61333/1062196890.parquet', 'kaggle/input/asl-signs/train_landmark_files/2044/215975047.parquet', 'kaggle/input/asl-signs/train_landmark_files/25571/1919433080.parquet', 'kaggle/input/asl-signs/train_landmark_files/61333/3652280161.parquet', 'kaggle/input/asl-signs/train_landmark_files/26734/1811603217.parquet']


In [179]:
X_train, y_train = sample_parquets(data_files, 1000)
X_train = process_data(X_train)

0
100
200
300
400
500


In [151]:
for x in X_train:
    if len(x)!= 32580:
        print("Length of x is not 32580", len(x))
    if len(x['frame'].unique()) != NUM_FRAMES:
        print("Number of frames is not 60", len(x['frame'].unique()))

print("Length of X_train", len(X_train))
print("Length of y_train", len(y_train))

Length of X_train 1000
Length of y_train 1000


In [177]:
def extract_coords_from_df(df):
    return np.asarray(df[['x', 'y']].fillna(0.0000001).values)

def normalize_coords(coords):
    column_norm = np.linalg.norm(coords, axis=0)
    return coords / column_norm


X_train = [normalize_coords(extract_coords_from_df(x)) for x in X_train]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [173]:
class NN_model(torch.nn.Module):
    def __init__(self, input_dims, output_classes=250):
        super(NN_model, self).__init__()
        self.linear1 = nn.Linear(input_dims, 4096)
        self.activation1 = nn.ReLU()
        self.linear2 = nn.Linear(4096, 2048)
        self.activation2 = nn.ReLU()
        self.linear3 = nn.Linear(2048, 1024)
        self.activation3 = nn.ReLU()
        self.linear4 = nn.Linear(1024, 512)
        self.activation4 = nn.ReLU()
        self.linear5 = nn.Linear(512, 250)
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.dropout(x)
        x = self.linear4(x)
        x = self.activation4(x)
        x = self.dropout(x)
        x = self.linear5(x)
        return x

model = NN_model(97740, 250)

In [176]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
                       
def train_model(model, X_train, y_train, criterion, optimizer, epochs, batch_size):
    loss_list = []
    train_acc_list = []
    for epoch in range(epochs):
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size, :]
            X_batch = torch.tensor(X_batch, dtype=torch.float32)
            y_batch = torch.tensor(y_batch, dtype=torch.long)
            
            X_batch = X_batch.view(-1, 97740)
            optimizer.zero_grad()
            
            output = model(X_batch)
            y_pred_train = torch.argmax(output, dim=1)

            y_batch = torch.argmax(y_batch, dim=1)
            print("GT", y_batch)
            print("PREDS", y_pred_train)
            train_acc = (y_pred_train == y_batch).float().mean()
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

            loss_list.append(loss.item())
            train_acc_list.append(train_acc)
        print(f'Epoch {epoch} Loss {loss.item()} Accuracy {train_acc}')
    return model, loss_list, train_acc_list

model, loss, train_acc = train_model(model, X_train, y_train, criterion, optimizer, 10, 32)

plt.plot(train_acc)
plt.legend(['train'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.savefig('train_acc.png')

plt.plot(loss)
plt.legend(['train'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.savefig('train_loss.png')


GT tensor([ 69, 154, 115, 150,  86, 189, 188, 144,  58, 225,  85, 196, 243, 196,
         73, 116, 212, 138, 113, 106,  85, 171,  16, 104, 191,   4, 235, 168,
        223,  94, 104, 221])
PREDS tensor([  4,   4,   4,   4,   4,   4,   4,   4,   4,   4,   4,   4, 183,   4,
          4,   4,   4,   4,   4,   4,   4,   4,   4,   4,   4,   4,   4,  78,
          4,   4,   4,   4])


KeyboardInterrupt: 